In [1]:
# Import necessary libraries
from IPython.display import Image
from IPython.display import Latex
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy import misc
from scipy import stats
import scipy.spatial.distance as dist
import PIL
from PIL import ImageDraw
plt.style.use("ggplot")


In [1]:
import cv2

image1 = cv2.imread('bac.jpg')
image=cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
se=cv2.getStructuringElement(cv2.MORPH_RECT , (5,5))
bg=cv2.morphologyEx(image, cv2.MORPH_DILATE, se)
out_gray=cv2.divide(image, bg, scale=255)
out_binary=cv2.threshold(out_gray, 0, 255, cv2.THRESH_OTSU )[1] 

cv2.imshow('binary', out_binary)  
# cv2.imwrite('binary.png',out_binary)

cv2.imshow('orignal', image1)  
# cv2.imwrite('gray.png',out_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
img = cv2.imread('bac.jpg')
# down_width = 600
# down_height = 500
# down_points = (down_width, down_height)
# img = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
img = np.mean(img, axis=2)

In [23]:
# Function for removing background noise
def remove_background_noise(img, dilations=1, kernel_size=3, thresh=250):
    kern = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    clean_image = cv2.dilate(img, kern, iterations=dilations)
    # Remove all pixels that are brighter than thresh
    return clean_image * (clean_image < thresh)   

# Create clean image
clean_image = remove_background_noise(img, dilations=1, kernel_size=3, thresh=250)
# misc.imsave("clean_image.jpg", clean_image)

cv2.imshow('img', clean_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
# Function to detect cell locations using Bonferroni corrected one sample t-test
def detect_cell_location(clean_image, step_size=20, alpha=0.05, population_mean=220):
    n = clean_image.shape[1]
    m = clean_image.shape[0]
    x_coordinates = []
    y_coordinates = []
    number_of_comparisons = np.round((n/step_size)*(m/step_size))
    bonferroni_alpha = 0.05 / float(number_of_comparisons)
    for y in xrange(0, m, step_size):
        for x in xrange(0, n, step_size):
            wind = clean_image[y:y+step_size, x:x+step_size].flatten()
            t_statistic, p_value = stats.ttest_1samp(wind, population_mean)
            if p_value == 0:
                x_coordinates.append(x)
                y_coordinates.append(y)
    return (x_coordinates, y_coordinates)

In [26]:
# Determine the pixel population mean
pop_mean = np.mean(clean_image)

# Extract cell locations
x_coordinates, y_coordinates = detect_cell_location(clean_image, population_mean=pop_mean)

NameError: name 'xrange' is not defined